In [117]:
%matplotlib inline
import numpy as np
import pandas as pd
#import seaborn as sns
#from keras import layers
#from tqdm import tqdm
#from keras.layers import Dense, Activation, Flatten, Convolution1D, Dropout,MaxPooling1D,GlobalAveragePooling1D
import matplotlib.pyplot as plt
#from keras.optimizers import SGD
#from keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler
#from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
import re

In [128]:
test = pd.read_csv('./test.csv')
train = pd.read_csv('./train.csv')
full_data = [test,train]

In [129]:
train['Embarked'] = train['Embarked'].fillna('S')
train['Age'] = train['Age'].fillna(train['Age'].mean())  
train['FamilySize'] = train['SibSp'] + train['Parch'] + 1
train['hasCabinLabel'] = pd.notnull(train['Cabin'])
#train['CategoricalFare'] = pd.qcut(train['Fare'], 4)

In [130]:
def get_title(name):
    title_search = re.search(' ([A-Za-z]+)\.', name)
    # If the title exists, extract and return it.
    if title_search:
        return title_search.group(1)
    return ""
# Create a new feature Title, containing the titles of passenger names
for dataset in full_data:
    dataset['Title'] = dataset['Name'].apply(get_title)
# Group all non-common titles into one single grouping "Rare"
for dataset in full_data:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')


deck = {"A": 1, "B": 2, "C": 3, "D": 4, "E": 5, "F": 6, "G": 7, "U": 8}

for dataset in full_data:
    dataset['Cabin'] = dataset['Cabin'].fillna("U0")
    dataset['Deck'] = dataset['Cabin'].map(lambda x: re.compile("([a-zA-Z]+)").search(x).group())
    dataset['Deck'] = dataset['Deck'].map(deck)
    dataset['Deck'] = dataset['Deck'].fillna(0)
    dataset['Deck'] = dataset['Deck'].astype(int)     

In [131]:
train = pd.get_dummies(train,columns=['Embarked','Sex','Title'])
target = train['Survived']
train = train.drop('Survived',axis=1)
train = train.drop(['Ticket','Name','Cabin','Deck'],axis=1)

In [132]:
features = train.values
labels = target.values
from sklearn.model_selection import train_test_split
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25, random_state = 42)

In [133]:
# Import the model we are using
from sklearn.ensemble import RandomForestClassifier
# Instantiate model with 1000 decision trees
rf = RandomForestClassifier(n_estimators = 1000, random_state = 42)
# Train the model on training data
rf.fit(train_features, train_labels);

predictions = rf.predict(test_features)
from sklearn.metrics import confusion_matrix,accuracy_score
accuracy_score(test_labels,predictions)

0.83856502242152464

In [127]:
train.head()

,PassengerId,Pclass,Age,SibSp,Parch,Fare,FamilySize,hasCabinLabel,Deck,Embarked_C,Embarked_Q,Embarked_S,Sex_female,Sex_male,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Rare
0,1,3,22.0,1,0,7.2500,2,False,8,0,0,1,0,1,0,0,1,0,0
1,2,1,38.0,1,0,71.2833,2,True,3,1,0,0,1,0,0,0,0,1,0
2,3,3,26.0,0,0,7.9250,1,False,8,0,0,1,1,0,0,1,0,0,0
3,4,1,35.0,1,0,53.1000,2,True,3,0,0,1,1,0,0,0,0,1,0
4,5,3,35.0,0,0,8.0500,1,False,8,0,0,1,0,1,0,0,1,0,0
